# Error Analysis on 8760 profiles (22 profiles)

In [2]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs\error_analysis'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

##UNCOMMENT WHICH PROFILE BEING ANALYZED 
x = 'load'
x2 = 'Load'

#x = 'solar'
#x2 = 'Solar_Gen'

#x = 'wind'
#x2 = 'Wind_Gen'

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs\error_analysis


## General Error Analysis

difference in values 
root mean square for each region
other analyses? 

## Load in data 
## Choose the profile to use by commenting / uncommenting code

In [18]:
############### GENERAL DATA LOAD ###############
# have to manually input CSV file name if not one of the 22 profiles
#x_segments = pd.read_csv('')
#x_hours = pd.read_csv('')
#x_name = ''

############### IPM APPROACH (2 profiles) ###############
# NORM
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_NORM.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_NORM.csv')
#x_name = 'norm'

# TOD before group
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_timeofday.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_timeofday.csv')
#x_name = 'timeofday'

############### SEQUENTIAL APPROACH (9 profiles) ###############
# 1-hr
x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_bind_8760hr.csv')
x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_bind_8760hr.csv')
x_name = 'seq_1hr'

# 2-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_2hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_2hr.csv')
#x_name = 'seq_2hr'

# 4-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_4hr.csv')
#x_name = 'seq_4hr'

# 8-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_8hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_8hr.csv')
#x_name = 'seq_8hr'

# 12-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_12hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_12hr.csv')
#x_name = 'seq_12hr'

# 24-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_24hr.csv')
#x_name = 'seq_24hr'

# 48-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_48hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_48hr.csv')
#x_name = 'seq_48hr'

# 120-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_120hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_120hr.csv')
#x_name = 'seq_120hr'

# 8760-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_bind_1hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_bind_1hr.csv')
#x_name = 'seq_8760hr'

############### ONE/TWO DAY APPROACH (7 profiles) ###############
# One day type, monthly, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_1daytype_monthly_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_1daytype_monthly_24hr.csv')
#x_name = '1daytype_month_24hr'

# One day type, season, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_1daytype_season_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_1daytype_season_24hr.csv')
#x_name = '1daytype_season_24hr'

# Two day type, monthly, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_monthly_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_monthly_24hr.csv')
#x_name = '2daytype_month_24hr'

# Two day type, season, 24 hrs#
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_season_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_season_24hr.csv')
#x_name = '2daytype_season_24hr'

# Two day type, monthly, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_month_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_month_4hr.csv')
#x_name = '2daytype_month_4hr'

# Two day type, bimonthly, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_bimonth_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_bimonth_4hr.csv')
#x_name = '2daytype_bimonth_4hr'

# Two day type, season, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_season_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_season_4hr.csv')
#x_name = '2daytype_season_4hr'

############### THREE DAY TYPE APPROACH (6 profiles) ###############
# Three day type, monthly, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_monthly_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_monthly_24hr.csv')
#x_name = '3daytype_month_24hr'

# Three day type, season, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_season_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_season_24hr.csv')
#x_name = '3daytype_season_24hr'

# Three day type, annual, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_annual_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_annual_24hr.csv')
#x_name = '3daytype_annual_24hr'

# Three day type, bimonthly, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_bimonth_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_bimonth_4hr.csv')
#x_name = '3daytype_bimonth_4hr'

# Three day type, season based, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_seasonbased_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_seasonbased_4hr.csv')
#x_name = '3daytype_seasonbased_4hr'

# Three day type, season, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_season_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_season_4hr.csv')
#x_name = '3daytype_season_4hr'

## Create outputs: total number of representative hours, RMS

In [19]:
#print(x_hours)

# use groupby region to count number of representative hours in each region
aggregations = {'Avg':'count'}
stat1 = x_segments.groupby(['Region'],as_index=False).agg(aggregations)
stat1 = stat1.rename(columns={'Avg':'Rep_Count'})
#stat1 = stat1[['Region','Count']].copy()

print(stat1)

      Region  Rep_Count
0   ERC_REST       8760
1   ERC_WEST       8760
2       FRCC       8760
3   MIS_AMSO       8760
4     MIS_AR       8760
..       ...        ...
58   WECC_WY       8760
59  WEC_BANC       8760
60  WEC_CALN       8760
61  WEC_LADW       8760
62  WEC_SDGE       8760

[63 rows x 2 columns]


In [20]:
# calculate RMSE 
stat2 = x_hours[['Region','R_Group','Hour_Tot',x2,'Avg']].copy()
stat2 = pd.merge(stat2,stat1,on='Region',how='left')
stat2 = stat2[['Region','R_Group','Rep_Count','Hour_Tot',x2,'Avg']]

#print(stat2)

stat2['Diff'] = stat2[x2] - stat2['Avg'] 
stat2['Square'] = stat2['Diff']**2
stat_reg = stat2.groupby(['Region','Avg','Hour_Tot'],as_index=False).agg({'Square' : sum})
stat_reg = stat_reg.rename(columns = {'Square':'Square_Sum'}) 
stat_reg['Mean'] = stat_reg['Square_Sum'] / stat_reg['Hour_Tot'] 
stat_reg['RMSE'] = stat_reg['Mean']**(1/2)
stat_reg = stat_reg.drop(columns={'Square_Sum','Mean'})
print(stat_reg)

stat3 = pd.merge(stat2,stat_reg,on=['Region','Avg','Hour_Tot'],how='left')
print(stat3)

stat3.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'_error.csv')
stat_reg.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'region_error.csv')

# RMSE tells us how spread out the data is over a line of best fit

          Region    Avg  Hour_Tot  RMSE
0       ERC_REST  26989         1   0.0
1       ERC_REST  27006         1   0.0
2       ERC_REST  27025         1   0.0
3       ERC_REST  27059         1   0.0
4       ERC_REST  27069         1   0.0
...          ...    ...       ...   ...
190326  WEC_SDGE   4434         1   0.0
190327  WEC_SDGE   4446         1   0.0
190328  WEC_SDGE   4465         1   0.0
190329  WEC_SDGE   4470         1   0.0
190330  WEC_SDGE   4479         1   0.0

[190331 rows x 4 columns]
          Region R_Group  Rep_Count  Hour_Tot   Load    Avg  Diff  Square  \
0       ERC_REST     ERC       8760         1  26989  26989     0       0   
1       ERC_REST     ERC       8760         1  27006  27006     0       0   
2       ERC_REST     ERC       8760         1  27025  27025     0       0   
3       ERC_REST     ERC       8760         1  27059  27059     0       0   
4       ERC_REST     ERC       8760         1  27069  27069     0       0   
...          ...     ...       

In [21]:
# descriptive statistics of entire data set 
stat2['Diff'].describe()

count    551880.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: Diff, dtype: float64

In [22]:
# descriptive statistics of only a single region
x_FRCC = stat2[stat2['R_Group']=='FRCC']
print('number of rows in dataset for testing =',x_FRCC.shape[0])

x_FRCC_stat = x_FRCC['Diff'].describe()
print(x_FRCC_stat)

number of rows in dataset for testing = 8760
count    8760.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Diff, dtype: float64
